<a href="https://colab.research.google.com/github/constructor-s/aps1080_winter_2021/blob/main/E0/E0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 0: Preliminaries

## A. Context: AI, Autonomy, Search, Data Science, Control

> AI is ultimately the science behind the engineering of autonomous machines. "AI" includes a variety of disparate tools, each of which have different kinds of scope.

> 1. What is an autonomous machine, and what does autonomy mean?

An autonomous machine is one that is able to make decisions without being explicitly programmed, an automony means the ability to make its own decisions.

> 2a. Classical AI techniques employ trees of decision constructs (if statements, etc.) to achieve some form of "intelligent"-like behavior. An example of this is the famous Eliza AI-based "psychotherapist". What do you feel are the strengths and weaknesses of such an approach? State and explain three such strengths and three weaknesses.

Strengths:

1. Deterministic and predictable responses
2. Easy-to-understand logic
3. Easy to program

Weaknesses:

1. Non-generalizable
2. Manual design (time-consuming)
3. Limited by knowledge of the designer

> 2b. Data Science includes signal processing, adaptive filters, supervised and unsupervised machine learning. These techniques solve the modeling problem from the perspective of the data. That is, they serve to classify signals (data), interpolate signals (data), or extract signal from noise. Comment on how these functions relate to artificial intelligence?

Classification allows a machine to understand the semantic category of the data presented. Interpolation allows a machine to infer future data from current observations. Extraction allows a machine to differentiate between meaningful data from noise.

> 2c. Control theory concerns the development of feedback laws (i.e., control laws) that strive to push a system (the "plant") to a desired state. It does this based on observation of essential characteristics of the plant (the state, or some possibly noisy or lossy observation of the state), and construction of an appropriate feedback signal. How does this tool compare with the prior two, relating to the development of an autonomous system? Explain the similarities and differences from a structural (high level) point of view.

Control theory is similar to AI in that it programs a machine to perform desirable tasks. However, control theory produces instructions based on an explicit physical model of the plant and environment, while in AI such a model is more abstracted as a statistical model.

## B. Practice

Consider the game of tic tac toe (TTT). Two individuals (i.e., an X-player, and an O-player) play on a 9-position board, alternately placing "X" or "O" markers on the board. A player wins when they have achieved three markers of their type, in a row (vertical, horizontal, diagonal).

Code a TTT player in the following manner:

1. Create a python class maintains the TTT board. It should have a reset method (to clear the TTT board), methods for setting the position of the board for each player, and a method to indicate whether the game has been won (returning who has won), or whether the game is at a draw.

2. Test this python class, by playing a two-human game of TTT.

3. Now you are to create a computer player. The goal of the computer player is to win if possible, or at worst, not lose. First think about the strategy that you need to codify, and then codify it.

4. Test your python class, by playing a human-vs-computer game of TTT.

In [ ]:
import numpy as np
from itertools import chain, product

class TTT:
    """
    A tic-tac-toe game. X moves first.
    """
    CLEAR = -1
    O = 0
    X = 1
    i2s = {-1: " ", 0: "O", 1: "X"}
    i2winner = {-1: "Draw", 0: "O", 1: "X", None: "In progress"}

    def __init__(self, board=None):
        self.reset(board)

    def reset(self, board=None):
        if board is None:
            self._board = np.full((3, 3), fill_value=TTT.CLEAR)
        else:
            self._board = board.copy()
        n = np.count_nonzero(self.board == TTT.CLEAR)
        self.next_mover = TTT.X if n%2==1 else TTT.O # X moves first

    @property
    def board(self):
        return self._board

    def set_move(self, r, c):
        assert (r, c) in self.get_legal_moves(), "Move is illegal"
        self._board[r, c] = self.next_mover
        self.next_mover = self.get_opponent(self.next_mover)
    
    @staticmethod
    def get_opponent(player):
        return 1 - player

    def get_legal_moves(self):
        """
        Retrn a list of legal moves. 
        If the game already has a winner or is drawn, returns empty array.
        """
        if self.get_winner() is not None:
            return np.array(())
        return np.array(np.where(self.board == TTT.CLEAR)).T

    def get_winner(self):
        """
        Returns
        -------
        int
            0 if O won, 1 if X won, -1 if draw, None if game has not finished
        """
        for row in chain(self.board, 
                         self.board.T, 
                         (np.diag(self.board), 
                          np.diag(np.fliplr(self.board)))
                         ):
            if (row != TTT.CLEAR).all():
                unique_vals = np.unique(row)
                if len(unique_vals) == 1:
                    return unique_vals.item()
        if (self.board == TTT.CLEAR).any():
            return None
        else:
            return -1
    
    def __str__(self):
        board_str = [[TTT.i2s[i] for i in row] for row in self.board]
        return "  0 1 2\n" + "\n  ─┼─┼─\n".join(f"{i} " + '│'.join(row) for i, row in enumerate(board_str)) + f"\nWinner: {TTT.i2winner[self.get_winner()]}"


In [ ]:
#%% Test this Python class

game = TTT()
rng = np.random.RandomState(0)
for i, j in product(range(3), range(3)):
    legal_moves = game.get_legal_moves()
    if len(legal_moves) > 0:
        move = legal_moves[rng.randint(len(legal_moves))]
        game.set_move(*move)
        print(game)
    else:
        break

# print(game)

  0 1 2
0  │ │ 
  ─┼─┼─
1  │ │X
  ─┼─┼─
2  │ │ 
Winner: In progress
  0 1 2
0 O│ │ 
  ─┼─┼─
1  │ │X
  ─┼─┼─
2  │ │ 
Winner: In progress
  0 1 2
0 O│ │ 
  ─┼─┼─
1  │X│X
  ─┼─┼─
2  │ │ 
Winner: In progress
  0 1 2
0 O│ │ 
  ─┼─┼─
1  │X│X
  ─┼─┼─
2 O│ │ 
Winner: In progress
  0 1 2
0 O│ │ 
  ─┼─┼─
1  │X│X
  ─┼─┼─
2 O│X│ 
Winner: In progress
  0 1 2
0 O│ │ 
  ─┼─┼─
1  │X│X
  ─┼─┼─
2 O│X│O
Winner: In progress
  0 1 2
0 O│ │X
  ─┼─┼─
1  │X│X
  ─┼─┼─
2 O│X│O
Winner: In progress
  0 1 2
0 O│ │X
  ─┼─┼─
1 O│X│X
  ─┼─┼─
2 O│X│O
Winner: O


In [ ]:
class TTTComputerPlayer:
    def __init__(self, game, my_player):
        self.game = game
        self.my_player = my_player

    def get_move(self):
        for move in game.get_legal_moves():
            simulate_game = TTT(game.board.copy())
            simulate_game.next_mover = self.my_player
            simulate_game.set_move(*move)
            # Find winning move
            if simulate_game.get_winner() == self.my_player:
                return move
            # If cannot find winning move, check if this is opponent's winning move
            simulate_game = TTT(game.board.copy())
            simulate_game.next_mover = simulate_game.get_opponent(self.my_player)
            simulate_game.set_move(*move)
            if simulate_game.get_winner() == simulate_game.get_opponent(self.my_player):
                return move
        return game.get_legal_moves()[0]

class TTTHumanPlayer:
    def __init__(self, game, my_player):
        self.game = game
        self.my_player = my_player

    def get_move(self):
        print(f"Your are {TTT.i2s[self.my_player]}; the current board is:")
        print(self.game)
        return [int(i) for i in input("Please enter 'row column':").split()]


In [ ]:
game = TTT()
player1 = TTTComputerPlayer(game, 1)
player2 = TTTHumanPlayer(game, 0)
while True:
    print(player1.get_move())
    game.set_move(*player1.get_move())
    print(game)
    if game.get_winner() is not None:
        break
    game.set_move(*player2.get_move())
    print(game)
    if game.get_winner() is not None:
        break

[0 0]
  0 1 2
0 X│ │ 
  ─┼─┼─
1  │ │ 
  ─┼─┼─
2  │ │ 
Winner: In progress
Your are O; the current board is:
  0 1 2
0 X│ │ 
  ─┼─┼─
1  │ │ 
  ─┼─┼─
2  │ │ 
Winner: In progress
Please enter 'row column':1 1
  0 1 2
0 X│ │ 
  ─┼─┼─
1  │O│ 
  ─┼─┼─
2  │ │ 
Winner: In progress
[0 1]
  0 1 2
0 X│X│ 
  ─┼─┼─
1  │O│ 
  ─┼─┼─
2  │ │ 
Winner: In progress
Your are O; the current board is:
  0 1 2
0 X│X│ 
  ─┼─┼─
1  │O│ 
  ─┼─┼─
2  │ │ 
Winner: In progress
Please enter 'row column':0 2
  0 1 2
0 X│X│O
  ─┼─┼─
1  │O│ 
  ─┼─┼─
2  │ │ 
Winner: In progress
[2 0]
  0 1 2
0 X│X│O
  ─┼─┼─
1  │O│ 
  ─┼─┼─
2 X│ │ 
Winner: In progress
Your are O; the current board is:
  0 1 2
0 X│X│O
  ─┼─┼─
1  │O│ 
  ─┼─┼─
2 X│ │ 
Winner: In progress
Please enter 'row column':1 0
  0 1 2
0 X│X│O
  ─┼─┼─
1 O│O│ 
  ─┼─┼─
2 X│ │ 
Winner: In progress
[1 2]
  0 1 2
0 X│X│O
  ─┼─┼─
1 O│O│X
  ─┼─┼─
2 X│ │ 
Winner: In progress
Your are O; the current board is:
  0 1 2
0 X│X│O
  ─┼─┼─
1 O│O│X
  ─┼─┼─
2 X│ │ 
Winner: In progress
Pl